# Querying your GPU with PyCUDA

我们要实现使用pycuda进行deviceQuery的设计

- intializing CUDA as follows:
```python
import pycuda.driver as drv
drv.init()
```
或者
```python
import pycuda.autoinit
```

In [29]:
import pycuda.driver as drv 
drv.init()
print(f'Detected {drv.Device.count()} CUDA Capable device(s)')

for i in range(drv.Device.count()):
    gpu_device = drv.Device(i)
    print ('Device {}: {}'.format(i, gpu_device.name()))
    compute_capability = gpu_device.compute_capability()
    print(f'Capability  {compute_capability}')
    print(f'Memory   {gpu_device.total_memory()//(1024**2)} MB')

    # gpu_device.get_attributes()是一个字典，但是他的key是Enumeration Type，不是字符串
    device_attributes = {}
    for k,v in gpu_device.get_attributes().items():
        device_attributes[str(k)] = v
    


    # for k,v in device_attributes.items():
    #     print(f'{k}: {v}')
    # print(device_attributes)

    # the number of multiprocessor
    num_mp = device_attributes['MULTIPROCESSOR_COUNT']
    print(f'the number of streaming multiprocessor: {num_mp}')


Detected 2 CUDA Capable device(s)
Device 0: NVIDIA TITAN RTX
Capability  (7, 5)
Memory   24205 MB
the number of streaming multiprocessor: 72
Device 1: NVIDIA TITAN RTX
Capability  (7, 5)
Memory   24205 MB
the number of streaming multiprocessor: 72


A GPU divides its individual cores up into lager units known as Streaming Multiprocessor(SM), which will each individuallyhave a particular number of CUDA cores, depending on the compute capability.

The number of cores per sm is not indicated directly by the gpu --- this is given to us implicitly by the compute capability. We will have to look up some techinical documents to determine the number of cores per sm.

In [33]:
import pycuda.driver as drv 
drv.init()
print(f'Detected {drv.Device.count()} CUDA Capable device(s)')

for i in range(drv.Device.count()):
    gpu_device = drv.Device(i)
    print ('Device {}: {}'.format(i, gpu_device.name()))
    compute_capability = gpu_device.compute_capability()
    compute_capability = f'{compute_capability[0]}.{compute_capability[1]}'
    print(f'Capability  {compute_capability}')
    print(f'Memory   {gpu_device.total_memory()//(1024**2)} MB')

    # gpu_device.get_attributes()是一个字典，但是他的key是Enumeration Type，不是字符串
    device_attributes = {}
    for k,v in gpu_device.get_attributes().items():
        device_attributes[str(k)] = v
    
    # the number of multiprocessor
    num_sm = device_attributes['MULTIPROCESSOR_COUNT']
    print(f'the number of streaming multiprocessor: {num_mp}')

    # 根据计算能力推断每个 SM 的核心数
    cores_per_sm = {
        '3.0': 192,  # Kepler
        '3.5': 192,
        '5.0': 128,  # Maxwell
        '5.2': 128,
        '6.0': 64,   # Pascal
        '6.1': 128,
        '7.0': 64,   # Volta
        '7.5': 64,   # Turing
        '8.0': 128,  # Ampere
        '8.6': 128
    }[compute_capability]

    # the number of cuda cores on device
    print(f'the total number of cuda cores: {cores_per_sm*num_sm}')

    for k,v in device_attributes.items():
        print(f'{k} : {v}')


Detected 2 CUDA Capable device(s)
Device 0: NVIDIA TITAN RTX
Capability  7.5
Memory   24205 MB
the number of streaming multiprocessor: 72
the total number of cuda cores: 4608
ASYNC_ENGINE_COUNT : 3
CAN_MAP_HOST_MEMORY : 1
CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM : 1
CLOCK_RATE : 1770000
COMPUTE_CAPABILITY_MAJOR : 7
COMPUTE_CAPABILITY_MINOR : 5
COMPUTE_MODE : DEFAULT
COMPUTE_PREEMPTION_SUPPORTED : 1
CONCURRENT_KERNELS : 1
CONCURRENT_MANAGED_ACCESS : 1
DIRECT_MANAGED_MEM_ACCESS_FROM_HOST : 0
ECC_ENABLED : 0
GENERIC_COMPRESSION_SUPPORTED : 0
GLOBAL_L1_CACHE_SUPPORTED : 1
GLOBAL_MEMORY_BUS_WIDTH : 384
GPU_OVERLAP : 1
HANDLE_TYPE_POSIX_FILE_DESCRIPTOR_SUPPORTED : 1
HANDLE_TYPE_WIN32_HANDLE_SUPPORTED : 0
HANDLE_TYPE_WIN32_KMT_HANDLE_SUPPORTED : 0
HOST_NATIVE_ATOMIC_SUPPORTED : 0
INTEGRATED : 0
KERNEL_EXEC_TIMEOUT : 1
L2_CACHE_SIZE : 6291456
LOCAL_L1_CACHE_SUPPORTED : 1
MANAGED_MEMORY : 1
MAXIMUM_SURFACE1D_LAYERED_LAYERS : 2048
MAXIMUM_SURFACE1D_LAYERED_WIDTH : 32768
MAXIMUM_SURFACE1D_WIDTH : 

# Using pycuda' gpuarray class

Like numpy, pycuda's gpuarray class plays an analogously prominent role within GPU programming in Python.

This class has all the features like numpy --- multidimentional vector/matrix/tensor shape structuring, array-slicing, array unraveling and overload operators for point-wise computations.

## transferring data to and from the gpu with gpuarray

pycuda covers all of the overhead of memory allocation, deallocation and data transfers with the `gpuarray` class.

与`numpy`类似，使用vector/matrix/tensor shape structure for data. `gpuarray` objects perform automatic cleanup based on the lifetime, so we do not have to worry about **freeing any GPU memory** stored in a `gpuarray` object when we are done with it.


1. contain host data in some form of numpy array
2. use the `gpuarray.to_gpu(host_data)` command to transfer this over to the gpu and create a new gpu array
3. after computatio within gpu, retrieve the gpu data into new host memory with the  `gpuarray.get` function

In [1]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
host_data = np.array([1,2,3,4,5], dtype=np.float32)

device_data = gpuarray.to_gpu(host_data)

device_data_x2 = device_data * 2
host_data_x2 = device_data_x2.get()

print(host_data_x2)


[ 2.  4.  6.  8. 10.]


需要显式注明numpy data type，因为 float type 与C/C++直接相关，且之后需要写 inline CUDA C。CUDA不使用double类型。

```python
host_data = np.array([1,2,3,4,5], dtype=np.float32)
```


## Basic pointwise arithmetic operations with gpuarray

We can use the overloaded python multiplication operator (*) to multiple each element in a `gpuarray` object by a scalr value. 

A pointwise operation is intrinsically parallelizable, and so when we use this operation on a gpuarray object pycuda is able to offload each multiplication opeation onto a single thread, rather than computing each multiplication in serial (numpy can use the advanced SSE instructions found in modern x86 chips for these computations, so in some cases the performance will be comparable to a gpu).

To be clear, **these pointwise operations performed on the GPU are in parallel** since the computation of one element is not dependent on the computation of any other element.


In [5]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
x_host = np.array([1,2,3], dtype=np.float32)
y_host = np.array([1,1,1], dtype=np.float32)
z_host = np.array([2,2,2], dtype=np.float32)

x_device = gpuarray.to_gpu(x_host)
y_device = gpuarray.to_gpu(y_host)
z_device = gpuarray.to_gpu(z_host)

print(x_host + y_host)

print((x_device + y_device).get())



[2. 3. 4.]
<class 'pycuda.gpuarray.GPUArray'>


### A speed test

In [4]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
from time import time

# * np.random refers to the random module in Numpy, and the second random is the specific function generating an array of random floating-point numbers uniformly distributed in the range [0,1)
host_data = np.array(np.random.random(5* 10**7), dtype=np.float32)

t1 = time()
host_data_2x = host_data * np.float32(2)
t2 = time()

print(f'total time to compute on CPU：{t2-t1} s')


device_data = gpuarray.to_gpu(host_data)


t1 = time()
device_data_2x = device_data * np.float32(2)
t2 = time()

from_device = device_data_2x.get()
print(f'total time to compute on GPU：{t2-t1} s')


print(f'Is the host computation same as the Device computation?:{np.allclose(from_device, host_data_2x)}')






total time to compute on CPU：0.17195940017700195 s
total time to compute on GPU：0.0046041011810302734 s
Is the host computation same as the Device computation?:True


When we execute these code twice, we first notice that the CPU computation time is about the same for each computation. Yet, we notice that the GPU computation time is far slower than CPU computation the first time we run this, and it becomes much faster in the subsequent run 
```bash
total time to compute on CPU：0.16499710083007812 s
total time to compute on GPU：0.28182029724121094 s
Is the host computation same as the Device computation?:True
```

```bash
total time to compute on CPU：0.16763687133789062 s
total time to compute on GPU：0.0035741329193115234 s
Is the host computation same as the Device computation?:True
```

Why is this? By the nature of the PyCUDA library, GPU code is often compiled and linked with Nvidia's `nvcc` compiler the first time it is run in a given python session; it is then cached and, if the code is called again, then it doesn't have to be recompiled.


*In PyCUDA, GPU code is often compiled at runtime with the NVIDIA nvcc compiler and then subsequently called from PyCUDA. This can lead to an unexpected slowdown, usually the first time a program or GPU operation is run in a given Python session.*

# Using PyCUDA's elementWisekernel for performing pointwise computations

We'll have to write a little bit of `inline code` in CUDA C, which is compiled externally by NVIDIA's `nvcc` compiler and then launched at runtime by our code via PyCUDA.

We use the term **kernel**. By kernel, we always mean a function that is launched directly onto the GPU by CUDA. We will use several functions from PyCUDA that **generate templates and design patterns** for different types of kernels, easing our transition into GPU programming.

In [12]:
import numpy as np
import pycuda.autoinit
from pycuda import gpuarray
from time import time
from pycuda.elementwise import ElementwiseKernel

host_data = np.array(np.random.random(5 * 10**7), dtype=np.float32)
'''
first line: C pointers to allocate memory on GPU
second line: define element-wise operation
third line: Note PyCUDA automatically sets up the integer index i. Whenwe use i as our index, ElementwiseKernel will automatically parallelize our calcularion over i among the many cores
forth line: give our piece of code its internal CUDA C kernel name. Since this refers to CUDA'C namespace and not Python's

'''
gpu_2x_ker = ElementwiseKernel(
    "float *in, float *out",
    "out[i] = 2*in[i];",
    "gpu_2x_ker"
)

def speedcomparison():
    t1 = time()
    host_data_2x = host_data * np.float32(2)
    t2 = time()

    print(f"total time to compute on CPU: {t2-t1} s")

    device_data = gpuarray.to_gpu(host_data)
    # allocate memory for output
    device_data_2x = gpuarray.empty_like(device_data)

    t1 = time()
    gpu_2x_ker(device_data, device_data_2x)
    t2 = time()

    from_device = device_data_2x.get()
    print(f"total time to compute on GPU {t2-t1} s")

    print(f'Is the host computation same as the Device computation?:{np.allclose(from_device, host_data_2x)}')


if __name__ == '__main__':
    speedcomparison()
    speedcomparison()




total time to compute on CPU: 0.2131824493408203 s
total time to compute on GPU 0.2850375175476074 s
Is the host computation same as the Device computation?:True
total time to compute on CPU: 0.17487001419067383 s
total time to compute on GPU 0.00023365020751953125 s
Is the host computation same as the Device computation?:True


To reduce the time for recompiling and utilize the cache at runtime, we execute `speedcomparison` twice.

When using PyCUDA, the first time we run a GPU kernel function that has CUDA C code written inside our Python script (inline), PyCUDA automatically compiles that CUDA code into executable code using the Nvidia CUDA compiler `nvcc`. The compilation only happens the first time the function is called. After the code is compiled, then it is cached and re-used for the remainder of a given Python session.

Note: I think the main reason is compilation versus interpretation.

We allocate empty memory on the GPU with the `gpuarray.empty_like` function. This acts as a plain `malloc` in C, allocating an array of the same size and data type as `device_data`, but without copying anything. 

## Mandelbroy revisited

In [ ]:
def gpu_mandelbrot(width, height, real_low, real_high, imag_low, imag_high, max_iters, upper_bound):
